In [24]:
import os
from pathlib import Path

notebook_path = os.getcwd()
check_path = Path(notebook_path, "../tests/quality_tests/check_esp32").resolve().absolute()

print("Notebook Path:", notebook_path)
print("check_path:", check_path)

Notebook Path: c:\develop\MyPython\micropython-stubs\scripts
check_path: C:\develop\MyPython\micropython-stubs\tests\quality_tests\check_esp32


In [73]:
import shutil

if os.path.exists(check_path / "typings"):
    shutil.rmtree(check_path / "typings")

os.chdir(notebook_path)
%pip install --target {check_path}/typings --no-user "../publish/micropython-stdlib-stubs" micropython-esp32-stubs
# %pip install --target {check_path}/typings --no-user micropython-esp32-stubs

Processing c:\develop\mypython\micropython-stubs\publish\micropython-stdlib-stubsNote: you may need to restart the kernel to use updated packages.

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached micropython_esp32_stubs-1.21.0.post4-py3-none-any.whl.metadata (3.2 kB)
Using cached micropython_esp32_stubs-1.21.0.post4-py3-none-any.whl (147 kB)
  Created wheel for micropython-stdlib-stubs: filename=micropython_stdlib_stubs-1.1.1a3-py3-none-any.whl size=121918 sha256=a58e4b00b24064de30b21a2029ff78a2038605e125990cf1f3bb13198b4e6908
  Stored in directory: c:\users\josverl\appdata\local\pip\cache\wheels\dc\bf\64\badaeca3b642752745d83261430f6134cd4a4216f273387121
Successfully built micropython-stdlib-s

In [76]:
if check_path.exists():
    print(f"Found {check_path}")
else:
    Exception(f"Path {check_path} not found")

for f in (
    "typings/sys.pyi",
    # below is only if the typings folder is checked by mypy - whichit should not do
    # "typings/stdlib/typing.pyi",
    # "typings/stdlib/_collections_abc.pyi",
    # "typings/stdlib/abc.pyi",
    # "typings/stdlib/types.pyi",
    # "typings/stdlib/typing_extensions.pyi",
    # "typings/stdlib/_typeshed",
    # "typings/stdlib/collections",
    # "typings/stdlib/sys",
    # "typings/stdlib/builtins.pyi",
):
    file = check_path / f
    if file.exists():
        print(f"Removing {f}")
        if file.is_file():
            file.unlink()
        elif file.is_dir():
            shutil.rmtree(file)
    else:
        print(f"File {f} not found")

Found C:\develop\MyPython\micropython-stubs\tests\quality_tests\check_esp32
Removing typings/sys.pyi


In [83]:
import subprocess


def run_mypy(path: Path):
    print(f"Running mypy in {path}")
    cmd = ["mypy", "-v", "./check_esp32.py"]
    cmd = ["mypy", "-v", "."]
    try:
        output = subprocess.run(
            cmd,
            check=True,
            cwd=path,
            capture_output=True,
            text=True,
        )
        print(output.stdout)
        print(output.stderr)
    except subprocess.CalledProcessError as e:
        print(e)


run_mypy(check_path)

Running mypy in C:\develop\MyPython\micropython-stubs\tests\quality_tests\check_esp32
Command '['mypy', '-v']' returned non-zero exit status 1.


In [64]:

# run mypy
!cd  {check_path}
# !mypy {check_path}
!mypy {check_path}/check_esp32.py

C:\develop\MyPython\micropython-stubs\tests\quality_tests\check_esp32\check_esp32.py:2: error: Cannot find implementation or library stub for module named "esp32"  [import-not-found]
C:\develop\MyPython\micropython-stubs\tests\quality_tests\check_esp32\check_esp32.py:2: note: See https://mypy.readthedocs.io/en/stable/running_mypy.html#missing-imports
C:\develop\MyPython\micropython-stubs\tests\quality_tests\check_esp32\check_esp32.py:14: error: Cannot find implementation or library stub for module named "machine"  [import-not-found]
Found 2 errors in 1 file (checked 1 source file)
